# Lab 8 - Integer Programming - BnB for MIP

Information on group members:

1) 156035, Kuba Czech <br>
2) 156045, Wojciech Nagórka

In [1]:
from pulp import *
import numpy as np
import pandas as pd
import copy
import math

1) Given is the below MIP problem. Note that the first 5 variables are of an integer type with specified upper bounds

In [2]:
def getProblem(relaxed = False):
    
    A = [
        [0,3,2,0,0,0,-3,-1,0,0],
        [1,1,0,2,0,0,0,-1,2,1],
        [0,0,2,-2,3,0,-2,2,1,0],
        [0,0,2,0,0,-1,0,0,0,1],
        [0,2,0,0,0,-2,0,0,0,1],
        [1,4,0,0,0,0,-3,6,2,0],
        [2,2,0,0,2,2,0,0,2,2],
        [0,0,3,0,-1,1,0,-1,0,1],
        [0,0,0,0,5,0,1,1,0,3],
        [2,-7,0,0,0,1,0,8,2,0]]
    b = [10,15,20,20,30,50,40,20,25,25]
    c = [5, 7, 5, 5, 5, 5, 7, 4, 9, 10]
    uB = [5, 8, 4, 5, 4, 5, 5, 3, 3, 3]
    
    problem = LpProblem(name="bnb-problem", sense=LpMaximize)
    
    ### 5 integers and 3 continuous (if relaxed, 8 cont.)
    cat = ['Integer' for i in range(5)] + ['Continuous' for i in range(5)]
    if relaxed: cat = ['Continuous' for i in range(5)] + ['Continuous' for i in range(5)]
        
    x = [LpVariable(name="x"+ str(i+1), lowBound=0, upBound=uB[i], cat = cat[i]) for i in range(10)]
    
    for r in range(10):
        expr = lpSum([x[j] * A[r][j] for j in range(10)])
        problem += LpConstraint(e=expr, sense = -1, name = "baseC"+str(r+1), rhs = b[r])
        
    obj_func = lpSum([x[j] * c[j] for j in range(10)])
    problem += obj_func
    
    return x, problem

x, P = getProblem()
print(P)

bnb-problem:
MAXIMIZE
5*x1 + 10*x10 + 7*x2 + 5*x3 + 5*x4 + 5*x5 + 5*x6 + 7*x7 + 4*x8 + 9*x9 + 0
SUBJECT TO
baseC1: 3 x2 + 2 x3 - 3 x7 - x8 <= 10

baseC2: x1 + x10 + x2 + 2 x4 - x8 + 2 x9 <= 15

baseC3: 2 x3 - 2 x4 + 3 x5 - 2 x7 + 2 x8 + x9 <= 20

baseC4: x10 + 2 x3 - x6 <= 20

baseC5: x10 + 2 x2 - 2 x6 <= 30

baseC6: x1 + 4 x2 - 3 x7 + 6 x8 + 2 x9 <= 50

baseC7: 2 x1 + 2 x10 + 2 x2 + 2 x5 + 2 x6 + 2 x9 <= 40

baseC8: x10 + 3 x3 - x5 + x6 - x8 <= 20

baseC9: 3 x10 + 5 x5 + x7 + x8 <= 25

baseC10: 2 x1 - 7 x2 + x6 + 8 x8 + 2 x9 <= 25

VARIABLES
0 <= x1 <= 5 Integer
x10 <= 3 Continuous
0 <= x2 <= 8 Integer
0 <= x3 <= 4 Integer
0 <= x4 <= 5 Integer
0 <= x5 <= 4 Integer
x6 <= 5 Continuous
x7 <= 5 Continuous
x8 <= 3 Continuous
x9 <= 3 Continuous



2) The below function returns None if the problem has no feasible solutions. Otherwise, it returns a tuple: objective function values and a vector of decision variables. 

In [3]:
def getSolution(x, problem):
    status = problem.solve()
    if problem.status != 1: 
        return None
    return problem.objective.value(), [_.value() for _ in x]

3) PuLP can solve MIP problems. Hence, the "relaxed" flag can be set to False. Solve the problem and analyze the obtained outcome.  

In [4]:
x, problem = getProblem(relaxed = False)
print(getSolution(x, problem))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Kuba/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/045d774c27da4538aa5f8e9b7ebd2af4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/045d774c27da4538aa5f8e9b7ebd2af4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 83 RHS
At line 94 BOUNDS
At line 105 ENDATA
Problem MODEL has 10 rows, 10 columns and 47 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 211.333 - 0.00 seconds
Cgl0004I processed model has 7 rows, 10 columns (5 integer (0 of which binary)) and 36 elements
Cbc0012I Integer solution of -206 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0012I Integer solution of -207 found by DiveCoeff

4) Now, compare this solution with the one obtained for the relaxed LP problem: 

In [5]:
x, problem = getProblem(relaxed = True)
print(getSolution(x, problem))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Kuba/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/e7fbb20cfa9e43419481404af5e4d17a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/e7fbb20cfa9e43419481404af5e4d17a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 73 RHS
At line 84 BOUNDS
At line 95 ENDATA
Problem MODEL has 10 rows, 10 columns and 47 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-3) rows, 10 (0) columns and 36 (-11) elements
0  Obj -0 Dual inf 64.49999 (10)
6  Obj 211.33333
Optimal - objective value 211.33333
After Postsolve, objective 211.33333, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 211.3333333 - 6 iterations time 0.002, Presolve 0.00
Option for

5) Your task is to implement the Branch and Bound Algorithm for solving MIP problems. You can use the PuLP library for solving the relaxed LP subproblems. <br>
<ul> 
    <li> Firstly, as a node selection policy, implement the default DFS-like strategy as shown in the lecture (generate both children in one iteration; prioritize the left children, i.e., associated with the "<=" constraint). As for the variable selection policy, take the one with the lowest index  (default, arbitrary selection). 
<li> Identify how many LP relaxed problems have to be solved to find the optimum. Note that such a number was reported to be 35 for the default policies. However, it may vary slightly for different solvers due to possible multiple sub-optima.
<li> Propose at least 2 new node and variable selection policies with the aim of minimizing the number of solver runs required to reach the optimum.  Try getting below 20. 
    </ul>

In [6]:
# Variant 1 - DFS + default selection policy
def BranchAndBound():
    x_rel, problem_rel = getProblem(relaxed = True)
    x, problem = getProblem(relaxed = False)
    bestObj, bestSol = getSolution(x_rel, problem_rel)

    to_visit = []
    visited = [bestSol]
    for xx, xx_rel, val in zip(x, x_rel, bestSol):
        if round(val, 0) != val and xx.cat == 'Integer':
            x_new, new_problem = getProblem(relaxed = True)
            for name, const in problem_rel.constraints.items():
                if (name, const) not in new_problem.constraints.items():
                    new_problem += (const, name)
                    
            for xx_new in x_new:
                if xx_new.name == xx_rel.name:
                    new_problem += (xx_new <= math.floor(val), "branching constraint #0 left")
            to_visit.append((new_problem, x_new))

            x_new, new_problem = getProblem(relaxed = True)
            for name, const in problem_rel.constraints.items():
                if (name, const) not in new_problem.constraints.items():
                    new_problem += (const, name)

            for xx_new in x_new:
                if xx_new.name == xx_rel.name:
                    new_problem += (xx_new >= math.ceil(val), "branching constraint #0 right")
            to_visit.append((new_problem, x_new))
            break

    bestObj = 0
    counter = 1
    nr_of_iterations = 1

    while len(to_visit) > 0:
        # Solving current problem
        curr_problem, curr_x = to_visit.pop(0)  
        sol = getSolution(curr_x, curr_problem)
        nr_of_iterations += 1
        if sol is None:
            visited.append(None)
            continue
        else:
            obj, solution = sol
        
        if solution in visited:
            continue
        visited.append(solution)

        # We cut the branch that performs poorly
        if obj < bestObj:
            continue
            
        # we have continous value when we should have integer - we branch
        for xx, xx_rel, val in zip(x, x_rel, solution):
            if round(val, 0) != val and xx.cat == 'Integer':
                x_new1, new_problem1 = getProblem(relaxed = True)
                for name, const in curr_problem.constraints.items():
                    if (name, const) not in new_problem1.constraints.items():
                        var, coeff = next(iter(const.items()))
                        const_sense = const.sense
                        rhs = abs(const.constant)

                        for new_var in x_new1:
                            if new_var.name == var.name:
                                if const_sense == 1:
                                    expr = (new_var * coeff >= rhs, name)
                                elif const_sense == 0:
                                    expr = (new_var * coeff == rhs, name)
                                elif const_sense == -1:
                                    expr = (new_var * coeff <= rhs, name)
                                new_problem1 += expr

                for xx_new in x_new1:
                    if xx_new.name == xx_rel.name:
                        new_problem1 += (xx_new <= math.floor(val), f"branching constraint #{counter} left")

                x_new2, new_problem2 = getProblem(relaxed = True)
                for name, const in curr_problem.constraints.items():
                    if (name, const) not in new_problem2.constraints.items():
                        var, coeff = next(iter(const.items()))
                        const_sense = const.sense
                        rhs = abs(const.constant)

                        for new_var in x_new2:
                            if new_var.name == var.name:
                                if const_sense == 1:
                                    expr = (new_var * coeff >= rhs, name)
                                elif const_sense == 0:
                                    expr = (new_var * coeff == rhs, name)
                                elif const_sense == -1:
                                    expr = (new_var * coeff <= rhs, name)
                                new_problem2 += expr

                for xx_new in x_new2:
                    if xx_new.name == xx_rel.name:
                        new_problem2 += (xx_new >= math.ceil(val), f"branching constraint #{counter} right")
                
                to_visit = [(new_problem1, x_new1), (new_problem2, x_new2)] + to_visit
                counter += 1
                break
        
        # If all values that should be integers are integers, then we check if this beats our current highest score
        if all((round(val, 0) == val and xx.cat == 'Integer') or (xx.cat == 'Continuous') for val, xx in zip(solution, x)):
            if obj > bestObj:
                bestObj = obj
                bestSol = solution
        
    print(nr_of_iterations)
    print(bestObj, bestSol)
    return bestObj, bestSol, nr_of_iterations

In [7]:
# Variant 2 - BFS + default selection policy
def BranchAndBound2():
    x_rel, problem_rel = getProblem(relaxed = True)
    x, problem = getProblem(relaxed = False)
    bestObj, bestSol = getSolution(x_rel, problem_rel)

    to_visit = []
    visited = [bestSol]
    for xx, xx_rel, val in zip(x, x_rel, bestSol):
        if round(val, 0) != val and xx.cat == 'Integer':
            x_new, new_problem = getProblem(relaxed = True)
            for name, const in problem_rel.constraints.items():
                if (name, const) not in new_problem.constraints.items():
                    new_problem += (const, name)
                    
            for xx_new in x_new:
                if xx_new.name == xx_rel.name:
                    new_problem += (xx_new <= math.floor(val), "branching constraint #0 left")
            to_visit.append((new_problem, x_new))

            x_new, new_problem = getProblem(relaxed = True)
            for name, const in problem_rel.constraints.items():
                if (name, const) not in new_problem.constraints.items():
                    new_problem += (const, name)

            for xx_new in x_new:
                if xx_new.name == xx_rel.name:
                    new_problem += (xx_new >= math.ceil(val), "branching constraint #0 right")
            to_visit.append((new_problem, x_new))
            break

    bestObj = 0
    counter = 1
    nr_of_iterations = 1

    while len(to_visit) > 0:
        # Solving current problem
        curr_problem, curr_x = to_visit.pop(0)  
        sol = getSolution(curr_x, curr_problem)
        nr_of_iterations += 1
        if sol is None:
            visited.append(None)
            continue
        else:
            obj, solution = sol
        
        if solution in visited:
            continue
        visited.append(solution)

        # We cut the branch that performs poorly
        if obj < bestObj:
            continue
            
        # we have continous value when we should have integer - we branch
        for xx, xx_rel, val in zip(x, x_rel, solution):
            if round(val, 0) != val and xx.cat == 'Integer':
                x_new1, new_problem1 = getProblem(relaxed = True)
                for name, const in curr_problem.constraints.items():
                    if (name, const) not in new_problem1.constraints.items():
                        var, coeff = next(iter(const.items()))
                        const_sense = const.sense
                        rhs = abs(const.constant)

                        for new_var in x_new1:
                            if new_var.name == var.name:
                                if const_sense == 1:
                                    expr = (new_var * coeff >= rhs, name)
                                elif const_sense == 0:
                                    expr = (new_var * coeff == rhs, name)
                                elif const_sense == -1:
                                    expr = (new_var * coeff <= rhs, name)
                                new_problem1 += expr

                for xx_new in x_new1:
                    if xx_new.name == xx_rel.name:
                        new_problem1 += (xx_new <= math.floor(val), f"branching constraint #{counter} left")

                x_new2, new_problem2 = getProblem(relaxed = True)
                for name, const in curr_problem.constraints.items():
                    if (name, const) not in new_problem2.constraints.items():
                        var, coeff = next(iter(const.items()))
                        const_sense = const.sense
                        rhs = abs(const.constant)

                        for new_var in x_new2:
                            if new_var.name == var.name:
                                if const_sense == 1:
                                    expr = (new_var * coeff >= rhs, name)
                                elif const_sense == 0:
                                    expr = (new_var * coeff == rhs, name)
                                elif const_sense == -1:
                                    expr = (new_var * coeff <= rhs, name)
                                new_problem2 += expr

                for xx_new in x_new2:
                    if xx_new.name == xx_rel.name:
                        new_problem2 += (xx_new >= math.ceil(val), f"branching constraint #{counter} right")
                
                to_visit = to_visit + [(new_problem1, x_new1), (new_problem2, x_new2)]
                counter += 1
                break
        
        # If all values that should be integers are integers, then we check if this beats our current highest score
        if all((round(val, 0) == val and xx.cat == 'Integer') or (xx.cat == 'Continuous') for val, xx in zip(solution, x)):
            if obj > bestObj:
                bestObj = obj
                bestSol = solution
        
    print(nr_of_iterations)
    print(bestObj, bestSol)
    return bestObj, bestSol, nr_of_iterations

In [8]:
# Variant 3 - DFS + policy choosing a variable with maximum number of constraints
def sortListElementWise(normalList, sortedList, listToSort):
    newList = [0 for i in range(10)]
    normalList = [i.name for i in normalList]
    sortedList = [i.name for i in sortedList]
    for i in range(len(sortedList)):
        index = normalList.index(sortedList[i])
        newList[i] = listToSort[index]
    return newList

def BranchAndBound3():
    x_rel, problem_rel = getProblem(relaxed = True)
    x, problem = getProblem(relaxed = False)
    bestObj, bestSol = getSolution(x_rel, problem_rel)

    to_visit = []
    visited = [bestSol]

    const_vars_rel = []
    # Counting constraints and sorting x_rel
    for name, const in problem_rel.constraints.items():
        for item in const.items():
            const_vars_rel.append(item[0])
    const_vars_names = [i.name for i in const_vars_rel]
    const_vars_count = [(i, const_vars_names.count(i.name)) for i in set(const_vars_rel)]
    const_vars_count.sort(key = lambda x: x[1], reverse= True)
    x_rel_sorted = [i[0] for i in const_vars_count]
    bestSol_sorted = sortListElementWise(x_rel, x_rel_sorted, bestSol)
    x_sorted = sortListElementWise(x_rel, x_rel_sorted, x)

    for xx, xx_rel, val in zip(x_sorted, x_rel_sorted, bestSol_sorted):
        if round(val, 0) != val and xx.cat == 'Integer':
            x_new, new_problem = getProblem(relaxed = True)
            for name, const in problem_rel.constraints.items():
                if (name, const) not in new_problem.constraints.items():
                    new_problem += (const, name)
                    
            for xx_new in x_new:
                if xx_new.name == xx_rel.name:
                    new_problem += (xx_new <= math.floor(val), "branching constraint #0 left")
            to_visit.append((new_problem, x_new))

            x_new, new_problem = getProblem(relaxed = True)
            for name, const in problem_rel.constraints.items():
                if (name, const) not in new_problem.constraints.items():
                    new_problem += (const, name)

            for xx_new in x_new:
                if xx_new.name == xx_rel.name:
                    new_problem += (xx_new >= math.ceil(val), "branching constraint #0 right")
            to_visit.append((new_problem, x_new))
            break

    bestObj = 0
    counter = 1
    nr_of_iterations = 1

    while len(to_visit) > 0:
        # Solving current problem
        curr_problem, curr_x = to_visit.pop(0)  
        sol = getSolution(curr_x, curr_problem)
        nr_of_iterations += 1
        if sol is None:
            visited.append(None)
            continue
        else:
            obj, solution = sol
        
        if solution in visited:
            continue
        visited.append(solution)

        # We cut the branch that performs poorly
        if obj < bestObj:
            continue
            
        const_vars_rel = []
        # Counting constraints and sorting x_rel
        for name, const in curr_problem.constraints.items():
            for item in const.items():
                const_vars_rel.append(item[0])
        const_vars_names = [i.name for i in const_vars_rel]
        const_vars_count = [(i, const_vars_names.count(i.name)) for i in set(const_vars_rel)]
        const_vars_count.sort(key = lambda x: x[1], reverse= True)
        x_rel_sorted = [i[0] for i in const_vars_count]
        solution_sorted = sortListElementWise(x_rel, x_rel_sorted, solution)
        x_sorted = sortListElementWise(x_rel, x_rel_sorted, x)

        # we have continous value when we should have integer - we branch
        for xx, xx_rel, val in zip(x_sorted, x_rel_sorted, solution_sorted):
            if round(val, 0) != val and xx.cat == 'Integer':
                x_new1, new_problem1 = getProblem(relaxed = True)
                for name, const in curr_problem.constraints.items():
                    if (name, const) not in new_problem1.constraints.items():
                        # new_const = copy.deepcopy(const)
                        var, coeff = next(iter(const.items()))
                        const_sense = const.sense
                        rhs = abs(const.constant)

                        for new_var in x_new1:
                            if new_var.name == var.name:
                                if const_sense == 1:
                                    expr = (new_var * coeff >= rhs, name)
                                elif const_sense == 0:
                                    expr = (new_var * coeff == rhs, name)
                                elif const_sense == -1:
                                    expr = (new_var * coeff <= rhs, name)
                                new_problem1 += expr

                for xx_new in x_new1:
                    if xx_new.name == xx_rel.name:
                        new_problem1 += (xx_new <= math.floor(val), f"branching constraint #{counter} left")

                x_new2, new_problem2 = getProblem(relaxed = True)
                for name, const in curr_problem.constraints.items():
                    if (name, const) not in new_problem2.constraints.items():
                        # new_const = copy.deepcopy(const)
                        var, coeff = next(iter(const.items()))
                        const_sense = const.sense
                        rhs = abs(const.constant)

                        for new_var in x_new2:
                            if new_var.name == var.name:
                                if const_sense == 1:
                                    expr = (new_var * coeff >= rhs, name)
                                elif const_sense == 0:
                                    expr = (new_var * coeff == rhs, name)
                                elif const_sense == -1:
                                    expr = (new_var * coeff <= rhs, name)
                                new_problem2 += expr

                for xx_new in x_new2:
                    if xx_new.name == xx_rel.name:
                        new_problem2 += (xx_new >= math.ceil(val), f"branching constraint #{counter} right")
                
                to_visit = [(new_problem1, x_new1), (new_problem2, x_new2)] + to_visit
                counter += 1
                break
        
        # If all values that should be integers are integers, then we check if this beats our current highest score
        if all((round(val, 0) == val and xx.cat == 'Integer') or (xx.cat == 'Continuous') for val, xx in zip(solution, x)):
            if obj > bestObj:
                bestObj = obj
                bestSol = solution
        
    print(nr_of_iterations)
    print(bestObj, bestSol)
    return bestObj, bestSol, nr_of_iterations

In [9]:
# Variant 4 - BFS + policy choosing a variable with maximum number of constraints
def BranchAndBound4():
    x_rel, problem_rel = getProblem(relaxed = True)
    x, problem = getProblem(relaxed = False)
    bestObj, bestSol = getSolution(x_rel, problem_rel)

    to_visit = []
    visited = [bestSol]

    const_vars_rel = []
    # Counting constraints and sorting x_rel
    for name, const in problem_rel.constraints.items():
        for item in const.items():
            const_vars_rel.append(item[0])
    const_vars_names = [i.name for i in const_vars_rel]
    const_vars_count = [(i, const_vars_names.count(i.name)) for i in set(const_vars_rel)]
    const_vars_count.sort(key = lambda x: x[1], reverse= True)
    x_rel_sorted = [i[0] for i in const_vars_count]
    bestSol_sorted = sortListElementWise(x_rel, x_rel_sorted, bestSol)
    x_sorted = sortListElementWise(x_rel, x_rel_sorted, x)
    
    for xx, xx_rel, val in zip(x_sorted, x_rel_sorted, bestSol_sorted):
        if round(val, 0) != val and xx.cat == 'Integer':
            x_new, new_problem = getProblem(relaxed = True)
            for name, const in problem_rel.constraints.items():
                if (name, const) not in new_problem.constraints.items():
                    new_problem += (const, name)
                    
            for xx_new in x_new:
                if xx_new.name == xx_rel.name:
                    new_problem += (xx_new <= math.floor(val), "branching constraint #0 left")
            to_visit.append((new_problem, x_new))

            x_new, new_problem = getProblem(relaxed = True)
            for name, const in problem_rel.constraints.items():
                if (name, const) not in new_problem.constraints.items():
                    new_problem += (const, name)

            for xx_new in x_new:
                if xx_new.name == xx_rel.name:
                    new_problem += (xx_new >= math.ceil(val), "branching constraint #0 right")
            to_visit.append((new_problem, x_new))
            break

    bestObj = 0
    counter = 1
    nr_of_iterations = 1

    while len(to_visit) > 0:
        # Solving current problem
        curr_problem, curr_x = to_visit.pop(0)  
        sol = getSolution(curr_x, curr_problem)
        nr_of_iterations += 1
        if sol is None:
            visited.append(None)
            continue
        else:
            obj, solution = sol
        
        if solution in visited:
            continue
        visited.append(solution)

        # We cut the branch that performs poorly
        if obj < bestObj:
            continue
            
        const_vars_rel = []
        # Counting constraints and sorting x_rel
        for name, const in curr_problem.constraints.items():
            for item in const.items():
                const_vars_rel.append(item[0])
        const_vars_names = [i.name for i in const_vars_rel]
        const_vars_count = [(i, const_vars_names.count(i.name)) for i in set(const_vars_rel)]
        const_vars_count.sort(key = lambda x: x[1], reverse= True)
        x_rel_sorted = [i[0] for i in const_vars_count]
        solution_sorted = sortListElementWise(x_rel, x_rel_sorted, solution)
        x_sorted = sortListElementWise(x_rel, x_rel_sorted, x)

        # we have continous value when we should have integer - we branch
        for xx, xx_rel, val in zip(x_sorted, x_rel_sorted, solution_sorted):
            if round(val, 0) != val and xx.cat == 'Integer':
                x_new1, new_problem1 = getProblem(relaxed = True)
                for name, const in curr_problem.constraints.items():
                    if (name, const) not in new_problem1.constraints.items():
                        var, coeff = next(iter(const.items()))
                        const_sense = const.sense
                        rhs = abs(const.constant)

                        for new_var in x_new1:
                            if new_var.name == var.name:
                                if const_sense == 1:
                                    expr = (new_var * coeff >= rhs, name)
                                elif const_sense == 0:
                                    expr = (new_var * coeff == rhs, name)
                                elif const_sense == -1:
                                    expr = (new_var * coeff <= rhs, name)
                                new_problem1 += expr

                for xx_new in x_new1:
                    if xx_new.name == xx_rel.name:
                        new_problem1 += (xx_new <= math.floor(val), f"branching constraint #{counter} left")

                x_new2, new_problem2 = getProblem(relaxed = True)
                for name, const in curr_problem.constraints.items():
                    if (name, const) not in new_problem2.constraints.items():
                        var, coeff = next(iter(const.items()))
                        const_sense = const.sense
                        rhs = abs(const.constant)

                        for new_var in x_new2:
                            if new_var.name == var.name:
                                if const_sense == 1:
                                    expr = (new_var * coeff >= rhs, name)
                                elif const_sense == 0:
                                    expr = (new_var * coeff == rhs, name)
                                elif const_sense == -1:
                                    expr = (new_var * coeff <= rhs, name)
                                new_problem2 += expr

                for xx_new in x_new2:
                    if xx_new.name == xx_rel.name:
                        new_problem2 += (xx_new >= math.ceil(val), f"branching constraint #{counter} right")
                
                to_visit = to_visit + [(new_problem1, x_new1), (new_problem2, x_new2)]
                counter += 1
                break
        
        # If all values that should be integers are integers, then we check if this beats our current highest score
        if all((round(val, 0) == val and xx.cat == 'Integer') or (xx.cat == 'Continuous') for val, xx in zip(solution, x)):
            if obj > bestObj:
                bestObj = obj
                bestSol = solution
        
    print(nr_of_iterations)
    print(bestObj, bestSol)
    return bestObj, bestSol, nr_of_iterations

In [10]:
BNB1 = BranchAndBound()
BNB2 = BranchAndBound2()
BNB3 = BranchAndBound3()
BNB4 = BranchAndBound4()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Kuba/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/038f22b654fd4303afbdd686d5ea43cb-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/038f22b654fd4303afbdd686d5ea43cb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 73 RHS
At line 84 BOUNDS
At line 95 ENDATA
Problem MODEL has 10 rows, 10 columns and 47 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-3) rows, 10 (0) columns and 36 (-11) elements
0  Obj -0 Dual inf 64.49999 (10)
6  Obj 211.33333
Optimal - objective value 211.33333
After Postsolve, objective 211.33333, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 211.3333333 - 6 iterations time 0.002, Presolve 0.00
Option for

In [11]:
print("Branch and Bound 1 (DFS + default policy):", BNB1)
print("Branch and Bound 2 (BFS + default policy):", BNB2)
print("Branch and Bound 3 (DFS + choosing variable with most constraints):", BNB3)
print("Branch and Bound 4 (BFS + choosing variable with most constraints):", BNB4)

Branch and Bound 1 (DFS + default policy): (207.0, [3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0, 3.0], 35)
Branch and Bound 2 (BFS + default policy): (207.0, [3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0, 3.0], 39)
Branch and Bound 3 (DFS + choosing variable with most constraints): (207.0, [3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0, 3.0], 19)
Branch and Bound 4 (BFS + choosing variable with most constraints): (207.0, [3.0, 6.0, 4.0, 1.0, 1.0, 5.0, 5.0, 3.0, 2.0, 3.0], 23)
